# Using Azure OpenAI GPT models

This sample demonstrates how to get started with Azure OpenAI Chat Completions using `AI Foundry SDK`.

In [6]:
%pip install azure-ai-projects --quiet
%pip install azure-ai-inference[prompts] --quiet
%pip install azure-identity --quiet
%pip install python-dotenv --quiet

# ! pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load the environment variables from the `.env` file to connect to the LLM model.

In [28]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [29]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

# For Serverless API or Managed Compute endpoints
client = ChatCompletionsClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    credential=AzureKeyCredential(os.getenv("AZURE_OPENAI_API_KEY")),
)

# project = AIProjectClient.from_connection_string(
#   conn_str=os.getenv("AI_FOUNDRY_PROJECT_CONNECTION_STRING"),
#   credential=DefaultAzureCredential(api_key=os.getenv("AZURE_OPENAI_API_KEY")))

# client = project.inference.get_chat_completions_client(api_version=os.getenv("AZURE_OPENAI_API_VERSION"))

messages=[
    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

response = client.complete(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    messages=messages
)

print("💬 ", response.choices[0].message.content)

💬  Oh sure, let me just pull out my magical time-telling abilities. Wait—nope, I don’t have a clock. Why don’t you check your phone, computer, or literally anything else that tells time? So resourceful!


Lets view the full JSON response.

In [ ]:
# Print the full response object
print(response.to_json())

Let's view the probabilities of each word.

In [ ]:
import numpy as np
from IPython.display import display, HTML

html_content = ""

for content in response.choices[0].logprobs.content:

    top_logprobs = content.top_logprobs

    for i, logprob in enumerate(top_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )

    html_content += f"<br>"
    
display(HTML(html_content))